In [1]:
class Entity:
    def __init__(self, name):
        self.name = name

    def __repr__(self):
        return f"Name: {self.name}"

class Hotel(Entity):
    sortParam = 'name'

    def __init__(self, name, roomAvl, location, rating, pricePr):
        super().__init__(name)
        self.roomAvl = roomAvl
        self.location = location
        self.rating = rating
        self.pricePr = pricePr

    def __lt__(self, other):
        return getattr(self, Hotel.sortParam) < getattr(other, Hotel.sortParam)

    @staticmethod
    def sortByName():
        Hotel.sortParam = 'name'

    @staticmethod
    def sortByRate():
        Hotel.sortParam = 'rating'

    @staticmethod
    def sortByRoomAvailable():
        Hotel.sortParam = 'roomAvl'

    def __repr__(self):
        return (f"HotelName: {self.name}\tRoom Available: {self.roomAvl}\t"
                f"Location: {self.location}\tRating: {self.rating}\tPrice Per Room: {self.pricePr}")


In [2]:
class User(Entity):
    def __init__(self, uname, uId, cost):
        super().__init__(uname)
        self.uId = uId
        self.cost = cost

    def __repr__(self):
        return f"UserName: {self.name}\tUserId: {self.uId}\tBooking Cost: {self.cost}"


In [3]:
import mysql.connector
from mysql.connector import Error

class Database:
    def __init__(self, host, user, password, database):
        try:
            self.connection = mysql.connector.connect(
                host=host,
                user=user,
                password=password,
                database=database
            )
            self.cursor = self.connection.cursor()
        except Error as e:
            print(f"Error connecting to MySQL: {e}")
    
    def create_tables(self):
        try:
            self.cursor.execute("""
                CREATE TABLE IF NOT EXISTS Hotels (
                    id INT AUTO_INCREMENT PRIMARY KEY,
                    name VARCHAR(255),
                    roomAvl INT,
                    location VARCHAR(255),
                    rating INT,
                    pricePr INT
                )
            """)
            
            self.cursor.execute("""
                CREATE TABLE IF NOT EXISTS Users (
                    id INT AUTO_INCREMENT PRIMARY KEY,
                    uname VARCHAR(255),
                    uId INT,
                    cost INT,
                    hotel_name VARCHAR(255)
                )
            """)
            self.connection.commit()
        except Error as e:
            print(f"Error creating tables: {e}")
    
    def insert_hotel(self, hotel):
        try:
            self.cursor.execute("""
                INSERT INTO Hotels (name, roomAvl, location, rating, pricePr) VALUES (%s, %s, %s, %s, %s)
            """, (hotel.name, hotel.roomAvl, hotel.location, hotel.rating, hotel.pricePr))
            self.connection.commit()
        except Error as e:
            print(f"Error inserting hotel data: {e}")
    
    def insert_user(self, user, hotel_name):
        try:
            self.cursor.execute("""
                INSERT INTO Users (uname, uId, cost, hotel_name) VALUES (%s, %s, %s, %s)
            """, (user.name, user.uId, user.cost, hotel_name))
            self.connection.commit()
        except Error as e:
            print(f"Error inserting user data: {e}")
    
    def fetch_hotels(self):
        try:
            self.cursor.execute("SELECT * FROM Hotels")
            return self.cursor.fetchall()
        except Error as e:
            print(f"Error fetching hotels: {e}")
            return []
    
    def fetch_users(self):
        try:
            self.cursor.execute("SELECT * FROM Users")
            return self.cursor.fetchall()
        except Error as e:
            print(f"Error fetching users: {e}")
            return []
    
    def close(self):
        if self.connection.is_connected():
            self.cursor.close()
            self.connection.close()


In [4]:
def PrintHotelData(hotels):
    for h in hotels:
        print(h)

def SortHotelByName(hotels):
    print("SORT BY NAME:")
    Hotel.sortByName()
    hotels.sort()
    PrintHotelData(hotels)
    print()

def SortHotelByRating(hotels):
    print("SORT BY RATING:")
    Hotel.sortByRate()
    hotels.sort()
    PrintHotelData(hotels)
    print()

def PrintHotelByCity(s, hotels):
    print(f"HOTELS FOR {s} LOCATION ARE:")
    hotelsByLoc = [h for h in hotels if h.location == s]
    PrintHotelData(hotelsByLoc)
    print()

def SortByRoomAvailable(hotels):
    print("SORT BY ROOM AVAILABLE:")
    Hotel.sortByRoomAvailable()
    hotels.sort()
    PrintHotelData(hotels)
    print()

def SaveAndPrintUserData(userName, userId, bookingCost, hotels, db):
    users = []
    for i in range(3):
        u = User(userName[i], userId[i], bookingCost[i])
        users.append(u)
    
    for i in range(len(users)):
        db.insert_user(users[i], hotels[i].name)
    
    print("USERS DATA FROM DATABASE:")
    user_data = db.fetch_users()
    for user in user_data:
        print(user)


In [5]:
def HotelManagement(userName, userId, hotelName, bookingCost, rooms, locations, ratings, prices, db):
    hotels = []
    for i in range(3):
        h = Hotel(hotelName[i], rooms[i], locations[i], ratings[i], prices[i]) # class hotel object
        hotels.append(h)
        db.insert_hotel(h)
    
    print()
    PrintHotelData(hotels)
    SortHotelByName(hotels)
    SortHotelByRating(hotels)
    PrintHotelByCity("Bangalore", hotels)
    SortByRoomAvailable(hotels)
    SaveAndPrintUserData(userName, userId, bookingCost, hotels, db)


In [6]:
if __name__ == '__main__':
    userName = ["U1", "U2", "U3"]
    userId = [2, 3, 4]
    hotelName = ["H1", "H2", "H3"]
    bookingCost = [1000, 1200, 1100]
    rooms = [4, 5, 6]
    locations = ["Bangalore", "Bangalore", "Mumbai"]
    ratings = [5, 5, 3]
    prices = [100, 200, 100]
    
    db = Database(host='localhost', user='root', password='Kavijk93@jai', database='pythondb')
    db.create_tables()
    
    try:
        HotelManagement(userName, userId, hotelName, bookingCost, rooms, locations, ratings, prices, db)
        
        print("HOTELS DATA FROM DATABASE:")
        hotel_data = db.fetch_hotels()
        for hotel in hotel_data:
            print(hotel)
    
    finally:
        db.close()



HotelName: H1	Room Available: 4	Location: Bangalore	Rating: 5	Price Per Room: 100
HotelName: H2	Room Available: 5	Location: Bangalore	Rating: 5	Price Per Room: 200
HotelName: H3	Room Available: 6	Location: Mumbai	Rating: 3	Price Per Room: 100
SORT BY NAME:
HotelName: H1	Room Available: 4	Location: Bangalore	Rating: 5	Price Per Room: 100
HotelName: H2	Room Available: 5	Location: Bangalore	Rating: 5	Price Per Room: 200
HotelName: H3	Room Available: 6	Location: Mumbai	Rating: 3	Price Per Room: 100

SORT BY RATING:
HotelName: H3	Room Available: 6	Location: Mumbai	Rating: 3	Price Per Room: 100
HotelName: H1	Room Available: 4	Location: Bangalore	Rating: 5	Price Per Room: 100
HotelName: H2	Room Available: 5	Location: Bangalore	Rating: 5	Price Per Room: 200

HOTELS FOR Bangalore LOCATION ARE:
HotelName: H1	Room Available: 4	Location: Bangalore	Rating: 5	Price Per Room: 100
HotelName: H2	Room Available: 5	Location: Bangalore	Rating: 5	Price Per Room: 200

SORT BY ROOM AVAILABLE:
HotelName: H1	